In [21]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [22]:
fn_results = "./data/results.parquet"

In [23]:
def get_total_results(fn_results: str) -> pd.DataFrame:
    """Get results aggregate over all periods

    Args:
        fn_results: name of file with hourly results
    """
    # settings for aggregation
    cols = [
        "nuclear",
        "renewable",
        "netStorage",
        "demand",
        "energyNotServed",
        "curtailNuclear",
        "curtailRenewable",
    ]
    idx = [
        "share_storage",
        "share_generation",
        "share_renewable",
        "curtailRenewableFirst",
    ]
    # get results, add indicator which technology is dispatched first, and aggregate
    df_annual = (
        pd.read_parquet(fn_results)
        .assign(
            curtailRenewableFirst=lambda df: df["costCurtailRenewable"]
            >= df["costCurtailNuclear"]
        )
        .groupby(idx, as_index=False)[cols]
        .sum()
    )
    return df_annual
df_annual = get_total_results(fn_results)

In [24]:
df = pd.read_parquet(fn_results)
df

,nuclear,renewable,netStorage,demand,energyNotServed,storageLevel,curtailNuclear,curtailRenewable,share_generation,share_renewable,share_storage,costCurtailNuclear,costCurtailRenewable,date
0,0.008783,0.000000,0.000000,0.008783,0.0,0.000000,0.002632,0.000000,1.0,0.0,0.0,1.0,0.0,2017-06-01 00:00:00
1,0.008826,0.000000,0.000000,0.008826,0.0,0.000000,0.002590,0.000000,1.0,0.0,0.0,1.0,0.0,2017-06-01 01:00:00
2,0.008933,0.000000,0.000000,0.008933,0.0,0.000000,0.002483,0.000000,1.0,0.0,0.0,1.0,0.0,2017-06-01 02:00:00
3,0.009378,0.000000,0.000000,0.009378,0.0,0.000000,0.002037,0.000000,1.0,0.0,0.0,1.0,0.0,2017-06-01 03:00:00
4,0.010975,0.000000,0.000000,0.010975,0.0,0.000000,0.000440,0.000000,1.0,0.0,0.0,1.0,0.0,2017-06-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12719515,0.000000,0.009558,0.000594,0.010597,0.0,0.001038,0.000000,NaN,1.5,1.0,0.2,0.0,1.0,2018-05-31 19:00:00
12719516,0.000000,0.010271,0.001031,0.010278,0.0,0.000007,0.000000,NaN,1.5,1.0,0.2,0.0,1.0,2018-05-31 20:00:00
12719517,0.000000,0.009544,0.000007,0.009544,0.0,0.000000,0.000000,0.000991,1.5,1.0,0.2,0.0,1.0,2018-05-31 21:00:00
12719518,0.000000,0.008962,0.000000,0.008962,0.0,0.000000,0.000000,0.001494,1.5,1.0,0.2,0.0,1.0,2018-05-31 22:00:00


In [25]:
df_annual.share_renewable.unique()

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 0.4])

In [44]:
# inputs
curtail_res_first = True
cost_res = 2
cost_nuc = 1
cost_sto = 10
cost_ens = 100
share_generation = 1
variable = "cost"


def get_plot_variable(
    cost_res: float,
    cost_nuc: float,
    cost_sto: float,
    cost_ens: float,
    share_generation: float,
    curtail_res_first=True,
):
    return (
        df_annual.query(f"share_generation == {share_generation}")
        .query(f"curtailRenewableFirst == {curtail_res_first}")
        .assign(
            cost=lambda df: (
                (df["nuclear"] + df["curtailNuclear"]) * cost_nuc
                + (df["renewable"] + df["curtailRenewable"]) * cost_res
                + df["energyNotServed"] * cost_ens
                + df["demand"] * df["share_storage"] * cost_sto
            )
        )
        .drop(["curtailRenewableFirst", "share_generation"], axis=1)
    ).round(1)


df_cost = get_plot_variable(cost_res=cost_res, cost_nuc=cost_nuc, cost_sto=cost_sto,
                            cost_ens=cost_ens, share_generation=share_generation)
df_cost

,share_storage,share_renewable,nuclear,renewable,netStorage,demand,energyNotServed,curtailNuclear,curtailRenewable,cost
1,0.0,0.0,92.3,0.0,0.0,100.0,7.7,7.7,0.0,866.0
3,0.0,0.1,82.2,10.0,0.0,100.0,7.8,7.8,0.0,892.2
5,0.0,0.2,71.0,20.0,0.0,100.0,9.0,9.0,0.0,1017.4
7,0.0,0.3,59.1,29.9,0.0,100.0,11.0,10.9,0.1,1229.2
9,0.0,0.4,47.4,39.1,0.0,100.0,13.5,12.6,0.9,1491.3
...,...,...,...,...,...,...,...,...,...,...
1333,0.2,0.6,9.2,53.0,0.0,100.0,0.0,30.8,7.0,360.0
1335,0.2,0.7,6.6,58.0,-0.0,100.0,0.0,23.4,12.0,370.0
1337,0.2,0.8,4.2,62.1,-0.0,100.0,0.0,15.8,17.9,380.0
1339,0.2,0.9,2.1,65.6,-0.0,100.0,0.0,7.9,24.4,390.0


In [67]:
df_plot = df_cost.copy()
df = df_plot.pivot_table(
    variable,
    "share_storage",
    "share_renewable",
)
print(df.values.shape, len(df.index), len(df.columns))
fig = px.imshow(
    df.values,
    x=df.columns,
    y=df.index,
    labels=dict(
        x="Renewable share [%]",
        y="Storage size [% of total demand]",
        color=f"{variable}",
    ),
    aspect="auto",
)
fig.show()

(3, 11) 3 11


In [63]:
df.values

array([[ 375.33333333,  200.        ,  280.        ],
       [ 390.73333333,  210.        ,  290.        ],
       [ 439.13333333,  220.        ,  300.        ],
       [ 516.4       ,  230.        ,  310.        ],
       [ 610.43333333,  240.        ,  320.        ],
       [ 713.93333333,  250.        ,  330.        ],
       [ 822.        ,  260.        ,  340.        ],
       [ 932.5       ,  270.        ,  350.        ],
       [1044.56666667,  280.        ,  360.        ],
       [1157.76666667,  290.        ,  370.        ],
       [1271.66666667,  300.        ,  380.        ]])

In [71]:
def plot_heatmap(df_plot: pd.DataFrame, variable: str = "cost") -> go.Figure:
    """Plot heat map for a given variable

    Args:
        df_plot: basic data for plotting already filtered for generation share
        variable: variable to plot
    """
    df = df_plot.pivot_table(
        variable,
        "share_storage",
        "share_renewable",
    )
    fig = px.imshow(
        df.values,
        x=list(df.columns),
        y=list(df.index),
        labels=dict(
            y="Renewable share [%]",
            x="Storage size [% of total demand]",
            color=f"{variable}",
        ),
        aspect="auto",
    )
    return fig


fig = plot_heatmap(df_cost)
fig.show()

In [57]:
df.index

Index([0.0, 0.1, 0.2, 0.3, 0.4000000000000001, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], dtype='float64', name='share_renewable')

In [31]:
fig = go.Figure(
    data=[go.Surface(z=df.values, x=df.index, y=df.columns)],
)
fig.update_layout(
    scene=dict(
        xaxis_title="x: Renewable share [%]",
        yaxis_title="y: Storage size [% of total demand]",
        zaxis_title=f"z: {variable}",
    ),
    autosize=True,
    width=500,
    height=500,
    margin=dict(l=65, r=50, b=65, t=90),
)
fig.show()

In [32]:
fig = px.imshow(
    df.values,
    x=df.index,
    y=df.columns,
    labels=dict(
        x="Renewable share [%]", 
        y="Storage size [% of total demand]", 
        color=f"{variable}"
        ),
    aspect="auto"
)
fig.show()

In [33]:
df

share_storage,0.00,0.02,0.04,0.06,0.08,0.10,0.12,0.14,0.16,0.18,0.20
share_renewable,,,,,,,,,,,
0.0,449.577104,130.0,150.0,170.0,190.0,210.0,230.0,250.0,270.0,290.0,310.0
0.1,488.894047,141.0,161.0,181.0,201.0,221.0,241.0,261.0,281.0,301.0,321.0
0.2,639.246736,152.0,172.0,192.0,212.0,232.0,252.0,272.0,292.0,312.0,332.0
0.3,854.617332,163.0,183.0,203.0,223.0,243.0,263.0,283.0,303.0,323.0,343.0
0.4,1119.491624,174.0,194.0,214.0,234.0,254.0,274.0,294.0,314.0,334.0,354.0
0.5,1421.781112,185.0,205.0,225.0,245.0,265.0,285.0,305.0,325.0,345.0,365.0
0.6,1748.058829,196.0,216.0,236.0,256.0,276.0,296.0,316.0,336.0,356.0,376.0
0.7,2085.385219,207.0,227.0,247.0,267.0,287.0,307.0,327.0,347.0,367.0,387.0
0.8,2429.207565,218.0,238.0,258.0,278.0,298.0,318.0,338.0,358.0,378.0,398.0
